Set up environment and load in data

In [ ]:
library(tidyverse)
library(brms)
library(here)
library(lemon)
theme_set(theme_bw())
helpers_path = paste0(here(),'/analysis/helpers/')
inputs_path = paste0(here(),'/inputs/')
source(paste0(helpers_path, '01_clean_behavioral_data.R'))
rm(data_bc_clean)
fig_out_path = paste0(here(), '/outputs/fig/')

Add columns that will be used for plots below.

In [ ]:
data_yn_clean = data_yn_clean %>%
  mutate(correct = ifelse(possiblePayoff>reference & yesChosen == 1, 1, ifelse(possiblePayoff < reference & yesChosen == 0, 1, 0))) %>%
  mutate(type_chr = ifelse(type == 1, "HT", "RE"),
         week = ifelse(week == 1, "Week 1", ifelse(week == 2, "Week 2", ifelse(week == 3, "Week 3", NA))),
         week = factor(week, levels = c("Week 1", "Week 2", "Week 3"))) %>%
  mutate(val_diff = possiblePayoff - reference,
         abs_val_diff = abs(possiblePayoff - reference),
         val_diff_bin = round(val_diff/50),
         val_diff_bin_str = paste0(val_diff_bin*50-25,":",val_diff_bin*50+25),
         val_diff_bin_str = factor(val_diff_bin_str, levels = c("-225:-175", "-175:-125", "-125:-75", "-75:-25", "-25:25", "25:75", "75:125", "125:175", "175:225")))

# Attribute effects 

## Correlation between attribtue values

Reminder: Are these regressors correlated?

Heatmaps of correlation of valueF, valueO, valueS, valueStim and valueDiff for each subject (and every run?)

In [ ]:
out = tibble()

for(cur_sub in unique(data_yn_clean$subnum)){
  
  # These correlations don't differ by day, session or reference amount
  # tmp = data_yn_clean %>% filter(subnum == cur_sub & day == cur_day)
  # tmp = data_yn_clean %>% filter(subnum == cur_sub & session == cur_day)
  # tmp = data_yn_clean %>% filter(subnum == cur_sub & reference == cur_day)
  
  tmp = data_yn_clean %>%
    filter(subnum == cur_sub) %>% 
    select(valueF, valueO, valueS, possiblePayoff)
  
  out = rbind(out, as_tibble(cor(tmp), rownames = "var1") %>%
                gather(var2, cor, -var1) %>%
                mutate(subnum = cur_sub))
  
}


p = out %>%  
  mutate(var1 = ifelse(var1 == "possiblePayoff", "valStim", var1),
         var2 = ifelse(var2 == "possiblePayoff", "valStim", var2)) %>%
  ggplot(aes(var1, var2, fill=cor, label = round(cor, 2)))+
  geom_tile()+
  geom_text()+
  scale_fill_gradient2(mid="#FBFEF9",low="#0C6291",high="#A63446", limits=c(-1,1))+
  scale_x_discrete(expand=c(0,0)) + 
  scale_y_discrete(expand=c(0,0))+
  labs(x="", y="")+
  facet_wrap(~subnum)+
  theme(legend.position = "bottom")


# ggsave(file=paste0(fig_out_path, 'yn_attrCors.jpg'), p, height = 5, width = 12, units="in")
p


## On accuracy

Accuracy for each attribute value

All stimuli

In [ ]:
data_yn_clean %>%
  group_by(subnum, valueO) %>%
  summarise(m_acc = mean(correct),
            sem_acc = sd(correct)/sqrt(n()),
            .groups = 'keep') %>%
  ggplot(aes(as.factor(valueO), m_acc))+
  geom_point()+
  geom_errorbar(aes(ymin = m_acc-sem_acc, ymax = m_acc+sem_acc), width = 0)+
  facet_wrap(~subnum)+
  labs(y = "Accuracy", x = "Orientation Value", title = "Y/N (All Stims)")+
  theme(panel.grid.minor.y = element_blank())

In [ ]:
data_yn_clean %>%
  filter(type == 0) %>%
  group_by(subnum, valueO) %>%
  summarise(m_acc = mean(correct),
            sem_acc = sd(correct)/sqrt(n()),
            .groups = 'keep') %>%
  ggplot(aes(as.factor(valueO), m_acc))+
  geom_point()+
  geom_errorbar(aes(ymin = m_acc-sem_acc, ymax = m_acc+sem_acc), width = 0)+
  facet_wrap(~subnum)+
  labs(y = "Accuracy", x = "Orientation Value", title = "Y/N (RE Stims)")+
  theme(panel.grid.minor.y = element_blank())

In [ ]:
data_yn_clean %>%
  filter(type == 0) %>%
  group_by(valueO, valueF) %>%
  summarise(m_acc = mean(correct),
            sem_acc = sd(correct)/sqrt(n()),
            .groups = 'keep') %>%
  ggplot(aes(as.factor(valueO), m_acc))+
  geom_point()+
  geom_errorbar(aes(ymin = m_acc-sem_acc, ymax = m_acc+sem_acc), width = 0)+
  facet_wrap(~valueF)+
  labs(y = "Accuracy", x = "Orientation Value", title = "Y/N (RE Stims)")+
  theme(panel.grid.minor.y = element_blank())

In [ ]:
fn_draws = paste0(inputs_path, 'yn_attrVal_cor_draws.csv')

if(file.exists(fn_draws)){
  yn_attr_cor_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_clean$subnum)
 
  yn_attr_cor_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub)
    
    cur_m <- brm(formula = correct ~  valueO * valueF * valueS,
                        data= cur_dat,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_attr_cor_draws = rbind(yn_attr_cor_draws, cur_draws)
  }
  names(yn_attr_cor_draws) = gsub(":", ".", names(yn_attr_cor_draws))
  write.csv(yn_attr_cor_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}
rm(subnums)

In [ ]:
names(as_tibble(yn_attr_cor_draws))[grepl("b_", names(as_tibble(yn_attr_cor_draws)))]

In [ ]:
yn_par_summary = as_tibble(yn_attr_cor_draws) %>%
  select(subnum, b_Intercept, b_valueO, b_valueF, b_valueS, b_valueO.valueF, b_valueO.valueS, b_valueF.valueS, b_valueO.valueF.valueS) %>%
  gather(key, value, -subnum) %>%
  mutate(key = recode(key,
                      "b_Intercept" = "Intercept",
                      "b_valueO" = "Orientation Value [-1, 1]", 
                      "b_valueF" = "Filling Value [-1, 1]", 
                      "b_valueS" = "Shape Value [-1, 1]", 
                      "b_valueO.valueF" = "Value O x F", 
                      "b_valueO.valueS" = "Value O x S", 
                      "b_valueF.valueS" = "Value F x S", 
                      "b_valueO.valueF.valueS" = "Value O x F x S"),
         key = factor(key, levels = c("Intercept", "Orientation Value [-1, 1]", "Filling Value [-1, 1]", "Shape Value [-1, 1]", "Value O x F", "Value O x S", "Value F x S", "Value O x F x S"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum))

In [ ]:
p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_wrap(~key, scales = 'free_x', nrow = 2) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - Correct ~ Orientation x Filling x Shape") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric(mid=0)

# ggsave(file=paste0(fig_out_path, 'yn_attrEffects_ParEsts.jpg'), p, height = 5, width = 9.5, units="in")

p

No main effect, no 3-way interaction but all 2-way interactions are significant.

How to interpret the 2-way interactions?  
It means that when the two attribute values aggree accuracy is higher. When they disaggree accuracy is lower.

Does this change by stim type?  
No.

In [ ]:
fn_draws = paste0(inputs_path, 'yn_attrValByType_cor_draws.csv')

if(file.exists(fn_draws)){
  yn_attr_cor_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_clean$subnum)
 
  yn_attr_cor_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(type))
    
    cur_m <- brm(formula = correct ~  valueO * valueF * valueS * type,
                        data= cur_dat,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_attr_cor_draws = rbind(yn_attr_cor_draws, cur_draws)
  }
  names(yn_attr_cor_draws) = gsub(":", ".", names(yn_attr_cor_draws))
  write.csv(yn_attr_cor_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}
rm(subnums)

In [ ]:
names(as_tibble(yn_attr_cor_draws))[grepl("b_", names(as_tibble(yn_attr_cor_draws)))]

In [ ]:
yn_par_summary = as_tibble(yn_attr_cor_draws) %>%
  select(subnum, b_Intercept, b_valueO, b_valueF, b_valueS, b_type1, b_valueO.valueF, b_valueO.valueS, b_valueF.valueS, b_valueO.type1, b_valueF.type1, b_valueS.type1, b_valueO.valueF.valueS, b_valueO.valueF.type1, b_valueO.valueS.type1, b_valueF.valueS.type1, b_valueO.valueF.valueS.type1) %>%
  gather(key, value, -subnum) %>%
  mutate(key = recode(key,
                      "b_Intercept" = "Intercept",
                      "b_valueO" = "Orientation Value [-1, 1]", 
                      "b_valueF" = "Filling Value [-1, 1]", 
                      "b_valueS" = "Shape Value [-1, 1]", 
                      "b_type1" = "Type (HT = 1, RE = 0)",
                      "b_valueO.type1" = "Value O x Type",
                      "b_valueF.type1" = "Value F x Type",
                      "b_valueS.type1" = "Value S x Type",
                      "b_valueO.valueF" = "Value O x F", 
                      "b_valueO.valueS" = "Value O x S", 
                      "b_valueF.valueS" = "Value F x S", 
                      "b_valueO.valueF.type1" = "Value O x F x Type", 
                      "b_valueO.valueS.type1" = "Value O x S x Type", 
                      "b_valueF.valueS.type1" = "Value F x S x Type",
                      "b_valueO.valueF.valueS" = "Value O x F x S",
                      "b_valueO.valueF.valueS.type1" = "Value O x F x S x Type"),
         key = factor(key, levels = c("Intercept", "Orientation Value [-1, 1]", "Filling Value [-1, 1]", "Shape Value [-1, 1]", "Type (HT = 1, RE = 0)", "Value O x Type", "Value F x Type", "Value S x Type", "Value O x F", "Value O x S", "Value F x S", "Value O x F x Type", "Value O x S x Type", "Value F x S x Type", "Value O x F x S", "Value O x F x S x Type"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum))

In [ ]:
p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_wrap(~key, scales = 'free_x', nrow = 2) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - Correct ~ Orientation x Filling x Shape x Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric(mid=0)

# ggsave(file=paste0(fig_out_path, 'yn_attrEffectsByStimType_ParEsts.jpg'), p, height = 5, width = 12, units="in")

p


## On "yes" choice

In [ ]:
names(data_yn_clean)

In [ ]:
fn_draws = paste0(inputs_path, 'yn_attrVal_yes_draws.csv')

if(file.exists(fn_draws)){
  yn_attr_yes_draws = read.csv(fn_draws)
} else{
  yn_attr_yes_draws = tibble()
  
  for(cur_sub in unique(data_yn_clean$subnum)){
  
    cur_dat = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub)
    
    cur_m <- brm(formula = yesChosen ~  valueO * valueF * valueS,
                        data= cur_dat,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_attr_yes_draws = rbind(yn_attr_yes_draws, cur_draws)
  }
  names(yn_attr_yes_draws) = gsub(":", ".", names(yn_attr_yes_draws))
  write.csv(yn_attr_yes_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}

In [ ]:
names(as_tibble(yn_attr_yes_draws))[grepl("b_", names(as_tibble(yn_attr_yes_draws)))]

In [ ]:
yn_par_summary = as_tibble(yn_attr_yes_draws) %>%
  select(subnum, b_Intercept, b_valueO, b_valueF, b_valueS, b_valueO.valueF, b_valueO.valueS, b_valueF.valueS, b_valueO.valueF.valueS) %>%
  gather(key, value, -subnum) %>%
  mutate(key = recode(key,
                      "b_Intercept" = "Intercept",
                      "b_valueO" = "Orientation Value [-1, 1]", 
                      "b_valueF" = "Filling Value [-1, 1]", 
                      "b_valueS" = "Shape Value [-1, 1]", 
                      "b_valueO.valueF" = "Value O x F", 
                      "b_valueO.valueS" = "Value O x S", 
                      "b_valueF.valueS" = "Value F x S", 
                      "b_valueO.valueF.valueS" = "Value O x F x S"),
         key = factor(key, levels = c("Intercept", "Orientation Value [-1, 1]", "Filling Value [-1, 1]", "Shape Value [-1, 1]", "Value O x F", "Value O x S", "Value F x S", "Value O x F x S"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum))

In [ ]:
p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_wrap(~key, scales = 'free_x', nrow = 2) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - Yes ~ Orientation x Filling x Shape") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric(mid=0)

# ggsave(file=paste0(fig_out_path, 'yn_attrEffectspYes_ParEsts.jpg'), p, height = 5, width = 9.5, units="in")

p

In [ ]:
fn_draws = paste0(inputs_path, 'yn_attrValByType_yes_draws.csv')

if(file.exists(fn_draws)){
  yn_attr_yes_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_clean$subnum)
 
  yn_attr_yes_draws = tibble()
  
  for(cur_sub in subnums){
  
    cur_dat = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(type))
    
    cur_m <- brm(formula = yesChosen ~  valueO * valueF * valueS * type,
                        data= cur_dat,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_attr_yes_draws = rbind(yn_attr_yes_draws, cur_draws)
  }
  names(yn_attr_yes_draws) = gsub(":", ".", names(yn_attr_yes_draws))
  write.csv(yn_attr_yes_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}
rm(subnums)

In [ ]:
names(as_tibble(yn_attr_yes_draws))[grepl("b_", names(as_tibble(yn_attr_yes_draws)))]

In [ ]:
yn_par_summary = as_tibble(yn_attr_yes_draws) %>%
  select(subnum, b_Intercept, b_valueO, b_valueF, b_valueS, b_type1, b_valueO.valueF, b_valueO.valueS, b_valueF.valueS, b_valueO.type1, b_valueF.type1, b_valueS.type1, b_valueO.valueF.valueS, b_valueO.valueF.type1, b_valueO.valueS.type1, b_valueF.valueS.type1, b_valueO.valueF.valueS.type1) %>%
  gather(key, value, -subnum) %>%
  mutate(key = recode(key,
                      "b_Intercept" = "Intercept",
                      "b_valueO" = "Orientation Value [-1, 1]", 
                      "b_valueF" = "Filling Value [-1, 1]", 
                      "b_valueS" = "Shape Value [-1, 1]", 
                      "b_type1" = "Type (HT = 1, RE = 0)",
                      "b_valueO.type1" = "Value O x Type",
                      "b_valueF.type1" = "Value F x Type",
                      "b_valueS.type1" = "Value S x Type",
                      "b_valueO.valueF" = "Value O x F", 
                      "b_valueO.valueS" = "Value O x S", 
                      "b_valueF.valueS" = "Value F x S", 
                      "b_valueO.valueF.type1" = "Value O x F x Type", 
                      "b_valueO.valueS.type1" = "Value O x S x Type", 
                      "b_valueF.valueS.type1" = "Value F x S x Type",
                      "b_valueO.valueF.valueS" = "Value O x F x S",
                      "b_valueO.valueF.valueS.type1" = "Value O x F x S x Type"),
         key = factor(key, levels = c("Intercept", "Orientation Value [-1, 1]", "Filling Value [-1, 1]", "Shape Value [-1, 1]", "Type (HT = 1, RE = 0)", "Value O x Type", "Value F x Type", "Value S x Type", "Value O x F", "Value O x S", "Value F x S", "Value O x F x Type", "Value O x S x Type", "Value F x S x Type", "Value O x F x S", "Value O x F x S x Type"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum))

In [ ]:
p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_wrap(~key, scales = 'free_x', nrow = 2) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - Yes ~ Orientation x Filling x Shape x Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric(mid=0)

# ggsave(file=paste0(fig_out_path, 'yn_attrEffectsByStimTypepYes_ParEsts.jpg'), p, height = 5, width = 12, units="in")

p


## On RT

In [ ]:
fn_draws = paste0(inputs_path, 'yn_attrVal_rt_draws.csv')

if(file.exists(fn_draws)){
  yn_attr_rt_draws = read.csv(fn_draws)
} else{

  yn_attr_rt_draws = tibble()
  
  for(cur_sub in unique(data_yn_clean$subnum)){
  
    cur_dat = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(log_rt = log(rt))
    
    cur_m <- brm(formula = log_rt ~  valueO * valueF * valueS,
                        data= cur_dat,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_attr_rt_draws = rbind(yn_attr_rt_draws, cur_draws)
  }
  names(yn_attr_rt_draws) = gsub(":", ".", names(yn_attr_rt_draws))
  write.csv(yn_attr_rt_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
yn_par_summary = as_tibble(yn_attr_rt_draws) %>%
  select(subnum, b_Intercept, b_valueO, b_valueF, b_valueS, b_valueO.valueF, b_valueO.valueS, b_valueF.valueS, b_valueO.valueF.valueS) %>%
  gather(key, value, -subnum) %>%
  mutate(key = recode(key,
                      "b_Intercept" = "Intercept",
                      "b_valueO" = "Orientation Value [-1, 1]", 
                      "b_valueF" = "Filling Value [-1, 1]", 
                      "b_valueS" = "Shape Value [-1, 1]", 
                      "b_valueO.valueF" = "Value O x F", 
                      "b_valueO.valueS" = "Value O x S", 
                      "b_valueF.valueS" = "Value F x S", 
                      "b_valueO.valueF.valueS" = "Value O x F x S"),
         key = factor(key, levels = c("Intercept", "Orientation Value [-1, 1]", "Filling Value [-1, 1]", "Shape Value [-1, 1]", "Value O x F", "Value O x S", "Value F x S", "Value O x F x S"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum))

In [ ]:
p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_wrap(~key, scales = 'free_x', nrow = 2) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - log(RT) ~ Orientation x Filling x Shape") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric(mid=0)

# ggsave(file=paste0(fig_out_path, 'yn_attrEffectsonRT_ParEsts.jpg'), p, height = 5, width = 9.5, units="in")

p

In [ ]:
fn_draws = paste0(inputs_path, 'yn_attrVal_rtByType_draws.csv')

if(file.exists(fn_draws)){
  yn_attr_rt_draws = read.csv(fn_draws)
} else{

  yn_attr_rt_draws = tibble()
  
  for(cur_sub in unique(data_yn_clean$subnum)){
  
    cur_dat = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(log_rt = log(rt),
             type = as.factor(type))
    
    cur_m <- brm(formula = log_rt ~  valueO * valueF * valueS*type,
                        data= cur_dat,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_attr_rt_draws = rbind(yn_attr_rt_draws, cur_draws)
  }
  names(yn_attr_rt_draws) = gsub(":", ".", names(yn_attr_rt_draws))
  write.csv(yn_attr_rt_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}


In [ ]:
yn_par_summary = as_tibble(yn_attr_rt_draws) %>%
  select(subnum, b_Intercept, b_valueO, b_valueF, b_valueS, b_type1, b_valueO.valueF, b_valueO.valueS, b_valueF.valueS, b_valueO.type1, b_valueF.type1, b_valueS.type1, b_valueO.valueF.valueS, b_valueO.valueF.type1, b_valueO.valueS.type1, b_valueF.valueS.type1, b_valueO.valueF.valueS.type1) %>%
  gather(key, value, -subnum) %>%
  mutate(key = recode(key,
                      "b_Intercept" = "Intercept",
                      "b_valueO" = "Orientation Value [-1, 1]", 
                      "b_valueF" = "Filling Value [-1, 1]", 
                      "b_valueS" = "Shape Value [-1, 1]", 
                      "b_type1" = "Type (HT = 1, RE = 0)",
                      "b_valueO.type1" = "Value O x Type",
                      "b_valueF.type1" = "Value F x Type",
                      "b_valueS.type1" = "Value S x Type",
                      "b_valueO.valueF" = "Value O x F", 
                      "b_valueO.valueS" = "Value O x S", 
                      "b_valueF.valueS" = "Value F x S", 
                      "b_valueO.valueF.type1" = "Value O x F x Type", 
                      "b_valueO.valueS.type1" = "Value O x S x Type", 
                      "b_valueF.valueS.type1" = "Value F x S x Type",
                      "b_valueO.valueF.valueS" = "Value O x F x S",
                      "b_valueO.valueF.valueS.type1" = "Value O x F x S x Type"),
         key = factor(key, levels = c("Intercept", "Orientation Value [-1, 1]", "Filling Value [-1, 1]", "Shape Value [-1, 1]", "Type (HT = 1, RE = 0)", "Value O x Type", "Value F x Type", "Value S x Type", "Value O x F", "Value O x S", "Value F x S", "Value O x F x Type", "Value O x S x Type", "Value F x S x Type", "Value O x F x S", "Value O x F x S x Type"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum))

In [ ]:
p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_wrap(~key, scales = 'free_x', nrow = 2) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - log(RT) ~ Orientation x Filling x Shape x Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric(mid=0)

# ggsave(file=paste0(fig_out_path, 'yn_attrEffectsonRTByStimType_ParEsts.jpg'), p, height = 5, width = 12, units="in")

p


# Reference amount effects 

## On accuracy

### Treating reference continously

Though discrete might be more appropriate

In [ ]:
names(data_yn_clean)

summary(data_yn_clean$reference)

In [ ]:
fn_draws = paste0(inputs_path, 'yn_refValAcc_draws.csv')

if(file.exists(fn_draws)){
  yn_refval_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_clean$subnum)
 
  yn_refval_draws = tibble()
  
  normMax = 1
  normMin = 0
  normMinRef = -1
  
  for(cur_sub in subnums){
  
    cur_dat = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(type),
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             norm_reference = (normMax - normMinRef) / (max(reference) - min(reference)) * (reference - max(reference)) + (normMax))
    
    cur_m <- brm(formula = correct ~  norm_reference * norm_day * type,
                        data= cur_dat,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_refval_draws = rbind(yn_refval_draws, cur_draws)
  }
  names(yn_refval_draws) = gsub(":", ".", names(yn_refval_draws))
  write.csv(yn_refval_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}
rm(subnums)

In [ ]:
names(as_tibble(yn_refval_draws))[grepl("b_", names(as_tibble(yn_refval_draws)))]

In [ ]:
yn_par_summary = as.tibble(yn_refval_draws) %>%
  select(subnum, b_Intercept, b_norm_reference, b_norm_day, b_type1, b_norm_reference.norm_day, b_norm_reference.type1, b_norm_day.type1, b_norm_reference.norm_day.type1) %>%
  gather(key, value, -subnum) %>%
  mutate(key = recode(key,
                      "b_Intercept" = "Intercept",
                     "b_norm_reference" = "Reference [-1, 1]",
                     "b_norm_day" = "Day [0, 1]",
                     "b_type1" = "Type (HT = 1, RE = 0)",
                     "b_norm_reference.norm_day" = "Reference x Day",
                     "b_norm_reference.type1" = "Reference x Type",
                     "b_norm_day.type1" = "Day x Type",
                     "b_norm_reference.norm_day.type1" = "Reference x Day x Type"),
         key = factor(key, levels = c("Intercept", "Reference [-1, 1]", "Day [0, 1]", "Type (HT = 1, RE = 0)","Reference x Day", "Reference x Type","Day x Type","Reference x Day x Type"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum))

In [ ]:
p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_wrap(~key, scales = 'free_x', nrow = 2) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - Correct ~ Reference x Day x Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric(mid=0)

# ggsave(file=paste0(fig_out_path, 'yn_attrEffectsByStimType_ParEsts.jpg'), p, height = 5, width = 12, units="in")

p


### Treating reference as discrete

In [ ]:
fn_draws = paste0(inputs_path, 'yn_refDiscValAcc_draws.csv')

if(file.exists(fn_draws)){
  yn_refval_draws = read.csv(fn_draws)
} else{
  subnums = unique(data_yn_clean$subnum)
 
  yn_refval_draws = tibble()
  
  normMax = 1
  normMin = 0
  normMinRef = -1
  
  for(cur_sub in subnums){
  
    cur_dat = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(type),
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             disc_reference = factor(reference, levels = c(0, -50, 50)))
    
    cur_m <- brm(formula = correct ~  disc_reference * norm_day * type,
                        data= cur_dat,
                        family = bernoulli(link = "logit"),
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_refval_draws = rbind(yn_refval_draws, cur_draws)
  }
  names(yn_refval_draws) = gsub(":", ".", names(yn_refval_draws))
  write.csv(yn_refval_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}
rm(subnums)

In [ ]:
names(as_tibble(yn_refval_draws))[grepl("b_", names(as_tibble(yn_refval_draws)))]

In [ ]:
yn_par_summary = as.tibble(yn_refval_draws) %>%
  select(subnum, b_Intercept, b_disc_referenceM50, b_disc_reference50, b_norm_day, b_type1, b_disc_referenceM50.norm_day, b_disc_reference50.norm_day, b_disc_referenceM50.type1, b_disc_reference50.type1, b_norm_day.type1, b_disc_referenceM50.norm_day.type1,b_disc_reference50.norm_day.type1 ) %>%
  gather(key, value, -subnum) %>%
  mutate(key = recode(key,
                      "b_Intercept" = "Intercept",
                     "b_disc_referenceM50" = "Reference 0 vs -50",
                     "b_disc_reference50" = "Reference 0 vs 50",
                     "b_norm_day"= "Day [0, 1]",
                     "b_type1" = "Type (HT = 1, RE = 0)",
                     "b_disc_referenceM50.norm_day" = "Reference 0 vs -50 x Day",
                     "b_disc_reference50.norm_day" = "Reference 0 vs 50 x Day",
                     "b_disc_referenceM50.type1" = "Reference 0 vs -50 x Type",
                     "b_disc_reference50.type1" = "Reference 0 vs 50 x Type",
                     "b_norm_day.type1"  = "Day x Type",
                     "b_disc_referenceM50.norm_day.type1" = "Reference 0 vs -50 x Day x Type",
                     "b_disc_reference50.norm_day.type1" = "Reference 0 vs 50 x Day x Type"),
         key = factor(key, levels = c("Intercept", "Reference 0 vs -50", "Reference 0 vs 50", "Day [0, 1]", "Type (HT = 1, RE = 0)", "Reference 0 vs -50 x Day", "Reference 0 vs 50 x Day", "Reference 0 vs -50 x Type", "Reference 0 vs 50 x Type", "Day x Type", "Reference 0 vs -50 x Day x Type", "Reference 0 vs 50 x Day x Type"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum))

In [ ]:
p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_wrap(~key, scales = 'free_x', nrow = 2) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - Correct ~ Reference x Day x Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric(mid=0)

# ggsave(file=paste0(fig_out_path, 'yn_refEffectsOnAcc_ParEsts.jpg'), p, height = 5, width = 12, units="in")

p


## Loss aversion

We're looking into any differences depending on reference amount to see if there is anything like "loss aversion" when the reference is -50.  

How would we operationalize "loss aversion" in this dataset?  

p(yes) for the same degree of value difference?  
Or even more specifically: p(yes) for when value difference is 0/small?  

This is basically looking at the intercept of the psychometric choice curves.

So the most comprehensive test you're looking for is probably: yes ~ value_diff x reference x type [x day]

In [ ]:
p = data_yn_clean %>%
  mutate(reference = factor(reference, levels = c(-50, 0, 50))) %>%
  group_by(subnum, val_diff_bin_str, type_chr, reference) %>%
  summarise(mean_y = mean(yesChosen),
            sem_y = sd(yesChosen)/sqrt(n()),
            .groups="keep") %>%
  mutate(type_chr = ifelse(type_chr == "HT", "Hyper-trained", "Regular Exposure")) %>%
  filter(type_chr == "Hyper-trained") %>%
ggplot() +
  geom_hline(aes(yintercept =.5), color="gray")+
  geom_vline(aes(xintercept = 5), color="gray")+
  geom_point(aes(val_diff_bin_str, mean_y, color = reference))+
  geom_line(aes(val_diff_bin_str, mean_y, color = reference, group = reference)) +
  geom_errorbar(aes(x = val_diff_bin_str, ymin = mean_y-sem_y, ymax = mean_y + sem_y, color = reference), width = 0, alpha = .6)+
  facet_grid(reference~subnum)+
  labs(x = "Value Stim - Value Reference", y = "p(Yes)", title="Y/N Task (HT)", alpha="", size = "")+
  theme(panel.grid = element_blank(),
        panel.border = element_rect(colour = "#1b9e77", linewidth = 2),
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Set1")+
  scale_x_discrete(labels=c("-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225"))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(color = "none") +
  annotate("rect", xmin = 4.75, xmax = 5.25, ymin = 0, ymax = 1,
           alpha = .5,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'yn_refEffectsOnPsychometricsHT.jpg'), p, height = 5, width = 12, units="in")

p

In [ ]:
p = data_yn_clean %>%
  mutate(reference = factor(reference, levels = c(-50, 0, 50))) %>%
  group_by(subnum, val_diff_bin_str, type_chr, reference) %>%
  summarise(mean_y = mean(yesChosen),
            sem_y = sd(yesChosen)/sqrt(n()),
            .groups="keep") %>%
  mutate(type_chr = ifelse(type_chr == "HT", "Hyper-trained", "Regular Exposure")) %>%
  filter(type_chr == "Regular Exposure") %>%
ggplot() +
  geom_hline(aes(yintercept =.5), color="gray")+
  geom_vline(aes(xintercept = 5), color="gray")+
  geom_point(aes(val_diff_bin_str, mean_y, color = reference))+
  geom_line(aes(val_diff_bin_str, mean_y, color = reference, group = reference)) +
  geom_errorbar(aes(x = val_diff_bin_str, ymin = mean_y-sem_y, ymax = mean_y + sem_y, color = reference), width = 0, alpha = .6)+
  facet_grid(reference~subnum)+
  labs(x = "Value Stim - Value Reference", y = "p(Yes)", title="Y/N Task (RE)", alpha="", size = "")+
  theme(panel.grid = element_blank(),
        panel.border = element_rect(colour = "#d95f02", linewidth = 2),
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Set1")+
  scale_x_discrete(labels=c("-225:-175", "", "-125:-75", "", "-25:25", "", "75:125", "", "175:225"))+
  scale_y_continuous(breaks = seq(0,1,.5))+
  guides(color = "none") +
  annotate("rect", xmin = 4.75, xmax = 5.25, ymin = 0, ymax = 1,
           alpha = .5,fill = "gray")

# ggsave(file=paste0(fig_out_path, 'yn_refEffectsOnPsychometricsRE.jpg'), p, height = 5, width = 12, units="in")

p

But since there aren't equal amounts of data in each value difference bin across the reference amounts (see above psychometric curves) you can test the values of specific interest more directly with 
data = value diff [-25:25]
yes ~ reference x type x day

In [ ]:
data_yn_clean %>%
  group_by(subnum, day, val_diff_bin_str, type_chr, reference) %>%
  summarise(mean_y = mean(yesChosen),
            sem_y = sd(yesChosen)/sqrt(n()),
            .groups="keep") %>%
  mutate(type_chr = ifelse(type_chr == "HT", "Hyper-trained", "Regular Exposure")) %>%
  filter(val_diff_bin_str == "-25:25") %>%
  ggplot(aes(day, mean_y, color = as.factor(reference)))+
  geom_point()+
  geom_errorbar(aes(ymin = mean_y - sem_y, ymax = mean_y + sem_y), width = .2) +
  geom_hline(aes(yintercept = 0.5), color = "gray") +
  facet_grid(type_chr ~ subnum)+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Set1")+
  labs(y = "p(Yes) for Value Difference [-25:25]", color = "Reference Amount", x = "Day")

Collapsing across days since there doesn't appear to be a change across visits.  

Loss aversion in this dataset: they would reject a sure loss (reference = -50) above chance when the value difference between the stim and the reference is small.  

**So test if p(yes) for -50 reference in small value difference trials is > .50.**


In [ ]:
p = data_yn_clean %>%
  mutate(time_bin = ifelse(day == 1, "Day 1", ifelse(day == 2, "Day 2", ifelse(day == 3, "Day 3", ifelse((day > 3) & (day < 8), "Week 2", "Week 3"))))) %>%
  group_by(subnum, val_diff_bin_str, type_chr, reference) %>%
  summarise(mean_y = mean(yesChosen),
            sem_y = sd(yesChosen)/sqrt(n()),
            .groups="keep") %>%
  mutate(type_chr = ifelse(type_chr == "HT", "Hyper-trained", "Regular Exposure")) %>%
  filter(val_diff_bin_str == "-25:25") %>%
  ggplot(aes(as.character(subnum), mean_y, color = as.factor(reference)))+
  geom_point(position = position_dodge(width=.2))+
  geom_errorbar(aes(ymin = mean_y - sem_y, ymax = mean_y + sem_y), position = position_dodge(width=.2),width = .2) +
  geom_hline(aes(yintercept = 0.5), color = "gray") +
  facet_wrap(~type_chr)+
  theme(panel.grid.major.y  = element_blank(),
        panel.grid.minor.y  = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Set1")+
  labs(y = "p(Yes) for Value Difference [-25:25]", color = "Reference Amount", x = "Subject")

# ggsave(file=paste0(fig_out_path, 'yn_refEffectsOnIntercepts.jpg'), p, height = 5, width = 6, units="in")

p

Based on the above graphs 4 subjects show loss aversion in both conditions, and this is larger in HT than RE.

The model below would compare p(yes) for different reference levels to each other. But I want to compare them to .5
data = value diff [-25:25]
yes ~ reference x type x day

What is the model for: **So test if p(yes) for -50 reference in small value difference trials is > .50.**. A binomial test?

In [ ]:
p = data_yn_clean %>%
    filter(val_diff_bin_str == "-25:25") %>%
  group_by(subnum, type_chr, reference) %>%
  summarise(sum_yes = sum(yesChosen),
            num_trials = n(),
            .groups="keep") %>%
  mutate(type_chr = ifelse(type_chr == "HT", "Hyper-trained", "Regular Exposure")) %>%
  mutate(bin_test_p = binom.test(sum_yes, num_trials, alternative = "greater")$p.value) %>%
  ggplot(aes(as.character(subnum), 1-bin_test_p, color = as.factor(reference)))+
  geom_point(position = position_dodge(width=.2), shape = 8, size = 3) +
  geom_hline(aes(yintercept = 0.95), color = "gray")+
  facet_wrap(~type_chr) +
  theme(panel.grid.major.y  = element_blank(),
        panel.grid.minor.y  = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Set1")+
  labs(y = "(1-p) for binomial test for p>.5", color = "Reference Amount", x = "Subject")

# ggsave(file=paste0(fig_out_path, 'yn_lossAversionBinomTest.jpg'), p, height = 5, width = 6, units="in")

p

Does accuracy in hard trials depend on the reference amount? Hard to detect a clear pattern consistent across subjects.

In [ ]:
data_yn_clean %>%
  group_by(subnum, day, val_diff_bin_str, type_chr, reference) %>%
  summarise(mean_y = mean(correct),
            sem_y = sd(correct)/sqrt(n()),
            .groups="keep") %>%
  mutate(type_chr = ifelse(type_chr == "HT", "Hyper-trained", "Regular Exposure")) %>%
  filter(val_diff_bin_str == "-25:25") %>%
  ggplot(aes(day, mean_y, color = as.factor(reference)))+
  geom_point()+
  geom_errorbar(aes(ymin = mean_y - sem_y, ymax = mean_y + sem_y), width = .2) +
  geom_hline(aes(yintercept = 0.5), color = "gray") +
  facet_grid(type_chr ~ subnum)+
  theme(panel.grid = element_blank(),
        legend.position = "bottom",
        axis.text.x = element_text(angle = 15, margin = margin(t = 5)),
        legend.box.margin=margin(t = -10),
        legend.spacing.y = unit(-0.25, "cm"),
        legend.box="vertical")+
  scale_color_brewer(palette = "Set1")+
  labs(y = "p(Correct) for Value Difference [-25:25]", color = "Reference Amount", x = "Day")

## On RT

In [ ]:
fn_draws = paste0(inputs_path, 'yn_refDiscValRT_draws.csv')

if(file.exists(fn_draws)){
  yn_refval_draws = read.csv(fn_draws)
} else{

  yn_refval_draws = tibble()
  
  normMax = 1
  normMin = 0

  for(cur_sub in unique(data_yn_clean$subnum)){
  
    cur_dat = data_yn_clean %>% 
      ungroup() %>%
      filter(subnum == cur_sub) %>%
      mutate(type = as.factor(type),
             norm_day = (normMax - normMin) / (max(day) - min(day)) * (day - max(day)) + (normMax),
             disc_reference = factor(reference, levels = c(0, -50, 50)),
             log_rt = log(rt))
    
    cur_m <- brm(formula = log_rt ~  disc_reference * norm_day * type,
                        data= cur_dat,
                        warmup = 500,
                        iter = 2500,
                        chains = 4,
                        init = "0",
                        cores = 4,
                        seed = 389253)
    
    cur_draws = as_draws_df(cur_m)
    cur_draws$subnum = cur_sub
    
    yn_refval_draws = rbind(yn_refval_draws, cur_draws)
  }
  names(yn_refval_draws) = gsub(":", ".", names(yn_refval_draws))
  write.csv(yn_refval_draws, fn_draws, row.names = F)

  rm(cur_dat, cur_m, cur_draws, cur_sub)
}

In [ ]:
names(as_tibble(yn_refval_draws))[grepl("b_", names(as_tibble(yn_refval_draws)))]

In [ ]:
yn_par_summary = as_tibble(yn_refval_draws) %>%
  select(subnum, b_Intercept, b_disc_referenceM50, b_disc_reference50, b_norm_day, b_type1, b_disc_referenceM50.norm_day, b_disc_reference50.norm_day, b_disc_referenceM50.type1, b_disc_reference50.type1, b_norm_day.type1, b_disc_referenceM50.norm_day.type1,b_disc_reference50.norm_day.type1 ) %>%
  gather(key, value, -subnum) %>%
  mutate(key = recode(key,
                      "b_Intercept" = "Intercept",
                     "b_disc_referenceM50" = "Reference 0 vs -50",
                     "b_disc_reference50" = "Reference 0 vs 50",
                     "b_norm_day"= "Day [0, 1]",
                     "b_type1" = "Type (HT = 1, RE = 0)",
                     "b_disc_referenceM50.norm_day" = "Reference 0 vs -50 x Day",
                     "b_disc_reference50.norm_day" = "Reference 0 vs 50 x Day",
                     "b_disc_referenceM50.type1" = "Reference 0 vs -50 x Type",
                     "b_disc_reference50.type1" = "Reference 0 vs 50 x Type",
                     "b_norm_day.type1"  = "Day x Type",
                     "b_disc_referenceM50.norm_day.type1" = "Reference 0 vs -50 x Day x Type",
                     "b_disc_reference50.norm_day.type1" = "Reference 0 vs 50 x Day x Type"),
         key = factor(key, levels = c("Intercept", "Reference 0 vs -50", "Reference 0 vs 50", "Day [0, 1]", "Type (HT = 1, RE = 0)", "Reference 0 vs -50 x Day", "Reference 0 vs 50 x Day", "Reference 0 vs -50 x Type", "Reference 0 vs 50 x Type", "Day x Type", "Reference 0 vs -50 x Day x Type", "Reference 0 vs 50 x Day x Type"))) %>%
  group_by(subnum, key) %>%
  summarise(.groups = 'keep',
            m = mean(value),
            l95 = HDInterval::hdi(value)[1],
            h95 = HDInterval::hdi(value)[2]) %>%
  mutate(subnum = as.factor(subnum))

In [ ]:
p = yn_par_summary %>%
  ggplot(aes(m, subnum)) +
  geom_vline(aes(xintercept = 0), color = "gray") + 
  geom_point(position = position_dodge(width=.75))+
  geom_errorbarh(aes(xmin = l95, xmax = h95, height=.2), position = position_dodge(width=.75))+
  facet_wrap(~key, scales = 'free_x', nrow = 2) +
  labs(x = "Parameter Estimate", y = "Subject", color = "", title = "95% HDI for posterior slopes - log(RT) ~ Reference x Day x Type") +
  theme(legend.position = "bottom",
        panel.grid = element_blank(),
        legend.box.margin=margin(t = -10))+
  scale_x_symmetric(mid=0)

# ggsave(file=paste0(fig_out_path, 'yn_refEffectsOnRT_ParEsts.jpg'), p, height = 5, width = 12, units="in")

p
